Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [4]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [5]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
batch_size = 128
hidden_nodes_size = 1024
# sizes = [1, 5, 10, 100, 500, 1024, 2000]

# for hidden_nodes_size in sizes:
print("Hidden nodes size =", hidden_nodes_size)
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

#   # Variables.
#   weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
#   biases = tf.Variable(tf.zeros([num_labels]))
#   output = tf.matmul(tf_train_dataset, weights) + biases + 100000*tf.nn.l2_loss(weights)
#   loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output, tf_train_labels))

  # Hidden Layer
  weights_hidden = weight_variable([image_size * image_size, hidden_nodes_size])
  biases_hidden = bias_variable([hidden_nodes_size])
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)

  # Output Layer
  weights_output = weight_variable([hidden_nodes_size, num_labels])
  biases_output = bias_variable([num_labels])
  output = tf.matmul(hidden, weights_output) + biases_output

  # Training computation.
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output, tf_train_labels)) + 0*tf.nn.l2_loss(weights_output)

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(output)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden),
                                             weights_output) + biases_output)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden),
                                             weights_output) + biases_output)
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  # print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

    if (step % 500 == 0):
      pred = accuracy(predictions, batch_labels)
      test = accuracy(valid_prediction.eval(), valid_labels)
      print("Step {},\tMinibatch accuracy: {:.1f},\tValidation accuracy: {:.1f}".format(str(step).rjust(4), pred, test))

  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Hidden nodes size = 1024
Step    0,	Minibatch accuracy: 11.7,	Validation accuracy: 29.8
Step  500,	Minibatch accuracy: 91.4,	Validation accuracy: 85.0
Step 1000,	Minibatch accuracy: 83.6,	Validation accuracy: 86.3
Step 1500,	Minibatch accuracy: 93.0,	Validation accuracy: 86.6
Step 2000,	Minibatch accuracy: 92.2,	Validation accuracy: 87.6
Step 2500,	Minibatch accuracy: 89.1,	Validation accuracy: 87.8
Step 3000,	Minibatch accuracy: 87.5,	Validation accuracy: 88.0
Test accuracy: 93.9%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [15]:
def myNetwork(beta, num_steps, interval):
    batch_size = 128
    hidden_nodes_size = 1024
    # sizes = [1, 5, 10, 100, 500, 1024, 2000]

    # for hidden_nodes_size in sizes:
    print("Hidden nodes size =", hidden_nodes_size)
    def weight_variable(shape):
      initial = tf.truncated_normal(shape, stddev=0.1)
      return tf.Variable(initial)

    def bias_variable(shape):
      initial = tf.constant(0.1, shape=shape)
      return tf.Variable(initial)

    graph = tf.Graph()
    with graph.as_default():
      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

    #   # Variables.
    #   weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    #   biases = tf.Variable(tf.zeros([num_labels]))
    #   output = tf.matmul(tf_train_dataset, weights) + biases + 100000*tf.nn.l2_loss(weights)
    #   loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output, tf_train_labels))

      # Hidden Layer
      weights_hidden = weight_variable([image_size * image_size, hidden_nodes_size])
      biases_hidden = bias_variable([hidden_nodes_size])
      hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)

      # Output Layer
      weights_output = weight_variable([hidden_nodes_size, num_labels])
      biases_output = bias_variable([num_labels])
      output = tf.matmul(hidden, weights_output) + biases_output

      # Training computation.
      loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output, tf_train_labels)) + \
                                beta*tf.nn.l2_loss(weights_output)

      # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(output)
      valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden),
                                                 weights_output) + biases_output)
      test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden),
                                                 weights_output) + biases_output)
#     num_steps = 3001

    with tf.Session(graph=graph) as session:
      tf.initialize_all_variables().run()
      # print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

        if (step % interval == 0):
          pred = accuracy(predictions, batch_labels)
          test = accuracy(valid_prediction.eval(), valid_labels)
          print("Step {},\tMinibatch accuracy: {:.1f},\tValidation accuracy: {:.1f}".format(str(step).rjust(4), pred, test))

      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

In [10]:
myNetwork(0)

Hidden nodes size = 1024
loss = Tensor("Mean:0", shape=(), dtype=float32)
Step    0,	Minibatch accuracy: 7.0,	Validation accuracy: 24.5
Step  500,	Minibatch accuracy: 90.6,	Validation accuracy: 85.5
Step 1000,	Minibatch accuracy: 85.2,	Validation accuracy: 86.2
Step 1500,	Minibatch accuracy: 93.0,	Validation accuracy: 87.2
Step 2000,	Minibatch accuracy: 93.0,	Validation accuracy: 87.3
Step 2500,	Minibatch accuracy: 89.8,	Validation accuracy: 87.6
Step 3000,	Minibatch accuracy: 89.8,	Validation accuracy: 88.1
Test accuracy: 94.1%


In [12]:
myNetwork(1)

Hidden nodes size = 1024
Step    0,	Minibatch accuracy: 14.8,	Validation accuracy: 26.5
Step  500,	Minibatch accuracy: 73.4,	Validation accuracy: 70.5
Step 1000,	Minibatch accuracy: 78.1,	Validation accuracy: 68.3
Step 1500,	Minibatch accuracy: 81.2,	Validation accuracy: 78.3
Step 2000,	Minibatch accuracy: 89.1,	Validation accuracy: 79.0
Step 2500,	Minibatch accuracy: 69.5,	Validation accuracy: 71.5
Step 3000,	Minibatch accuracy: 78.9,	Validation accuracy: 78.4
Test accuracy: 85.2%


In [13]:
myNetwork(10)

Hidden nodes size = 1024
Step    0,	Minibatch accuracy: 10.2,	Validation accuracy: 2.9
Step  500,	Minibatch accuracy: 9.4,	Validation accuracy: 10.0
Step 1000,	Minibatch accuracy: 8.6,	Validation accuracy: 10.0
Step 1500,	Minibatch accuracy: 11.7,	Validation accuracy: 10.0
Step 2000,	Minibatch accuracy: 6.2,	Validation accuracy: 10.0
Step 2500,	Minibatch accuracy: 10.2,	Validation accuracy: 10.0
Step 3000,	Minibatch accuracy: 7.8,	Validation accuracy: 10.0
Test accuracy: 10.0%


In [17]:
myNetwork(0.001, 3001, 500)

Hidden nodes size = 1024
Step    0,	Minibatch accuracy: 12.5,	Validation accuracy: 23.4
Step  500,	Minibatch accuracy: 89.8,	Validation accuracy: 85.3
Step 1000,	Minibatch accuracy: 85.2,	Validation accuracy: 86.6
Step 1500,	Minibatch accuracy: 93.8,	Validation accuracy: 87.3
Step 2000,	Minibatch accuracy: 91.4,	Validation accuracy: 87.2
Step 2500,	Minibatch accuracy: 89.8,	Validation accuracy: 87.9
Step 3000,	Minibatch accuracy: 89.8,	Validation accuracy: 88.2
Test accuracy: 94.3%


In [18]:
myNetwork(0.0001, 3001, 500)

Hidden nodes size = 1024
Step    0,	Minibatch accuracy: 10.2,	Validation accuracy: 23.3
Step  500,	Minibatch accuracy: 89.1,	Validation accuracy: 84.8
Step 1000,	Minibatch accuracy: 83.6,	Validation accuracy: 86.1
Step 1500,	Minibatch accuracy: 91.4,	Validation accuracy: 87.2
Step 2000,	Minibatch accuracy: 93.0,	Validation accuracy: 87.5
Step 2500,	Minibatch accuracy: 90.6,	Validation accuracy: 87.7
Step 3000,	Minibatch accuracy: 89.8,	Validation accuracy: 87.9
Test accuracy: 94.0%


In [16]:
myNetwork(0, 3001, 500)

Hidden nodes size = 1024
Step    0,	Minibatch accuracy: 8.6,	Validation accuracy: 32.2
Step  500,	Minibatch accuracy: 88.3,	Validation accuracy: 85.2
Step 1000,	Minibatch accuracy: 83.6,	Validation accuracy: 86.2
Step 1500,	Minibatch accuracy: 93.0,	Validation accuracy: 86.0
Step 2000,	Minibatch accuracy: 93.8,	Validation accuracy: 87.3
Step 2500,	Minibatch accuracy: 88.3,	Validation accuracy: 87.5
Step 3000,	Minibatch accuracy: 85.2,	Validation accuracy: 87.6
Test accuracy: 93.8%


In [29]:
myNetwork(0, 50, 5)

Hidden nodes size = 1024
Step    0,	Minibatch accuracy: 11.7,	Validation accuracy: 31.7
Step    5,	Minibatch accuracy: 55.5,	Validation accuracy: 66.0
Step   10,	Minibatch accuracy: 74.2,	Validation accuracy: 72.5
Step   15,	Minibatch accuracy: 71.1,	Validation accuracy: 76.3
Step   20,	Minibatch accuracy: 80.5,	Validation accuracy: 79.0
Step   25,	Minibatch accuracy: 73.4,	Validation accuracy: 78.4
Step   30,	Minibatch accuracy: 86.7,	Validation accuracy: 78.3
Step   35,	Minibatch accuracy: 75.0,	Validation accuracy: 79.3
Step   40,	Minibatch accuracy: 72.7,	Validation accuracy: 74.8
Step   45,	Minibatch accuracy: 78.9,	Validation accuracy: 73.9
Test accuracy: 84.4%


In [35]:
myNetwork(0.01, 50, 5)

Hidden nodes size = 1024
Step    0,	Minibatch accuracy: 10.9,	Validation accuracy: 16.9
Step    5,	Minibatch accuracy: 52.3,	Validation accuracy: 64.2
Step   10,	Minibatch accuracy: 72.7,	Validation accuracy: 74.4
Step   15,	Minibatch accuracy: 74.2,	Validation accuracy: 77.5
Step   20,	Minibatch accuracy: 79.7,	Validation accuracy: 78.2
Step   25,	Minibatch accuracy: 71.9,	Validation accuracy: 77.0
Step   30,	Minibatch accuracy: 80.5,	Validation accuracy: 78.1
Step   35,	Minibatch accuracy: 76.6,	Validation accuracy: 78.9
Step   40,	Minibatch accuracy: 75.8,	Validation accuracy: 77.3
Step   45,	Minibatch accuracy: 84.4,	Validation accuracy: 76.6
Test accuracy: 86.6%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [51]:
def myNetwork2(beta, num_steps, interval):
    batch_size = 128
    hidden_nodes_size = 1024
    # sizes = [1, 5, 10, 100, 500, 1024, 2000]

    # for hidden_nodes_size in sizes:
    print("Hidden nodes size =", hidden_nodes_size)
    def weight_variable(shape):
      initial = tf.truncated_normal(shape, stddev=0.1)
      return tf.Variable(initial)

    def bias_variable(shape):
      initial = tf.constant(0.1, shape=shape)
      return tf.Variable(initial)

    graph = tf.Graph()
    with graph.as_default():
      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

    #   # Variables.
    #   weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    #   biases = tf.Variable(tf.zeros([num_labels]))
    #   output = tf.matmul(tf_train_dataset, weights) + biases + 100000*tf.nn.l2_loss(weights)
    #   loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output, tf_train_labels))

      # Hidden Layer
      weights_hidden = weight_variable([image_size * image_size, hidden_nodes_size])
      biases_hidden = bias_variable([hidden_nodes_size])
      dropped_train = tf.nn.dropout(tf_train_dataset, 0.9)  
      hidden = tf.nn.relu(tf.matmul(dropped_train, weights_hidden) + biases_hidden)

      # Output Layer
      weights_output = weight_variable([hidden_nodes_size, num_labels])
      biases_output = bias_variable([num_labels])
      output = tf.matmul(hidden, weights_output) + biases_output

      # Training computation.
      loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output, tf_train_labels)) + \
                                beta*tf.nn.l2_loss(weights_output)

      # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(output)
      valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden),
                                                 weights_output) + biases_output)
      test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden),
                                                 weights_output) + biases_output)
#     num_steps = 3001

    with tf.Session(graph=graph) as session:
      tf.initialize_all_variables().run()
      # print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

        if (step % interval == 0):
          pred = accuracy(predictions, batch_labels)
          test = accuracy(valid_prediction.eval(), valid_labels)
          print("Step {},\tMinibatch accuracy: {:.1f},\tValidation accuracy: {:.1f}".format(str(step).rjust(4), pred, test))

      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

In [20]:
myNetwork2(0.01, 50, 5)

Hidden nodes size = 1024
Step    0,	Minibatch accuracy: 7.0,	Validation accuracy: 32.1
Step    5,	Minibatch accuracy: 49.2,	Validation accuracy: 63.3
Step   10,	Minibatch accuracy: 75.0,	Validation accuracy: 77.4
Step   15,	Minibatch accuracy: 78.9,	Validation accuracy: 77.0
Step   20,	Minibatch accuracy: 78.9,	Validation accuracy: 75.3
Step   25,	Minibatch accuracy: 70.3,	Validation accuracy: 79.3
Step   30,	Minibatch accuracy: 82.0,	Validation accuracy: 79.1
Step   35,	Minibatch accuracy: 75.0,	Validation accuracy: 79.5
Step   40,	Minibatch accuracy: 74.2,	Validation accuracy: 75.6
Step   45,	Minibatch accuracy: 78.9,	Validation accuracy: 72.4
Test accuracy: 86.2%


In [54]:
myNetwork2(0.01, 500, 50)

Hidden nodes size = 1024
Step    0,	Minibatch accuracy: 14.1,	Validation accuracy: 30.2
Step   50,	Minibatch accuracy: 81.2,	Validation accuracy: 79.2
Step  100,	Minibatch accuracy: 82.0,	Validation accuracy: 81.6
Step  150,	Minibatch accuracy: 76.6,	Validation accuracy: 78.2
Step  200,	Minibatch accuracy: 82.8,	Validation accuracy: 81.3
Step  250,	Minibatch accuracy: 85.9,	Validation accuracy: 82.6
Step  300,	Minibatch accuracy: 75.8,	Validation accuracy: 81.6
Step  350,	Minibatch accuracy: 85.2,	Validation accuracy: 82.9
Step  400,	Minibatch accuracy: 85.2,	Validation accuracy: 83.6
Step  450,	Minibatch accuracy: 86.7,	Validation accuracy: 84.2
Test accuracy: 90.0%


In [55]:
myNetwork2(0.01, 3001, 500)

Hidden nodes size = 1024
Step    0,	Minibatch accuracy: 8.6,	Validation accuracy: 22.8
Step  500,	Minibatch accuracy: 86.7,	Validation accuracy: 84.0
Step 1000,	Minibatch accuracy: 85.2,	Validation accuracy: 84.0
Step 1500,	Minibatch accuracy: 90.6,	Validation accuracy: 85.8
Step 2000,	Minibatch accuracy: 93.8,	Validation accuracy: 86.2
Step 2500,	Minibatch accuracy: 86.7,	Validation accuracy: 86.1
Step 3000,	Minibatch accuracy: 85.2,	Validation accuracy: 86.2
Test accuracy: 92.8%


In [56]:
myNetwork2(0.01, 3001, 500)

Hidden nodes size = 1024
Step    0,	Minibatch accuracy: 11.7,	Validation accuracy: 34.7
Step  500,	Minibatch accuracy: 86.7,	Validation accuracy: 83.5
Step 1000,	Minibatch accuracy: 85.2,	Validation accuracy: 84.4
Step 1500,	Minibatch accuracy: 89.1,	Validation accuracy: 85.7
Step 2000,	Minibatch accuracy: 91.4,	Validation accuracy: 85.7
Step 2500,	Minibatch accuracy: 85.9,	Validation accuracy: 86.4
Step 3000,	Minibatch accuracy: 85.2,	Validation accuracy: 86.2
Test accuracy: 92.8%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [73]:
def myNetwork3(beta, num_steps, interval):
    batch_size = 128
    hidden_nodes_size = 1024
    # sizes = [1, 5, 10, 100, 500, 1024, 2000]

    # for hidden_nodes_size in sizes:
    print("Hidden nodes size =", hidden_nodes_size)
    def weight_variable(shape):
      initial = tf.truncated_normal(shape, stddev=0.1)
      return tf.Variable(initial)

    def bias_variable(shape):
      initial = tf.constant(0.1, shape=shape)
      return tf.Variable(initial)

    graph = tf.Graph()
    with graph.as_default():
      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

    #   # Variables.
    #   weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    #   biases = tf.Variable(tf.zeros([num_labels]))
    #   output = tf.matmul(tf_train_dataset, weights) + biases + 100000*tf.nn.l2_loss(weights)
    #   loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output, tf_train_labels))

      # Hidden Layer
      weights_hidden = weight_variable([image_size * image_size, hidden_nodes_size])
      biases_hidden = bias_variable([hidden_nodes_size])
      dropped_train = tf.nn.dropout(tf_train_dataset, 0.9)  
      hidden = tf.nn.relu(tf.matmul(dropped_train, weights_hidden) + biases_hidden)

      # Output Layer
      weights_output = weight_variable([hidden_nodes_size, num_labels])
      biases_output = bias_variable([num_labels])
      output = tf.matmul(hidden, weights_output) + biases_output

      # Training computation.
      loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output, tf_train_labels)) + \
                                beta*tf.nn.l2_loss(weights_output)

      # Optimizer.
      #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
      global_step = tf.Variable(0)  # count the number of steps taken.
      learning_rate = tf.train.exponential_decay(0.5, global_step, num_steps, 0.96)
      optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)        
        
      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(output)
      valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden),
                                                 weights_output) + biases_output)
      test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden),
                                                 weights_output) + biases_output)
#     num_steps = 3001

    with tf.Session(graph=graph) as session:
      tf.initialize_all_variables().run()
      # print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

        if (step % interval == 0):
          pred = accuracy(predictions, batch_labels)
          test = accuracy(valid_prediction.eval(), valid_labels)
          print("Step {},\tMinibatch accuracy: {:.1f},\tValidation accuracy: {:.1f}".format(str(step).rjust(4), pred, test))

      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

In [58]:
myNetwork3(0.01, 3001, 500)

Hidden nodes size = 1024
Step    0,	Minibatch accuracy: 7.8,	Validation accuracy: 26.9
Step  500,	Minibatch accuracy: 87.5,	Validation accuracy: 83.6
Step 1000,	Minibatch accuracy: 84.4,	Validation accuracy: 84.7
Step 1500,	Minibatch accuracy: 92.2,	Validation accuracy: 85.7
Step 2000,	Minibatch accuracy: 92.2,	Validation accuracy: 85.8
Step 2500,	Minibatch accuracy: 87.5,	Validation accuracy: 86.2
Step 3000,	Minibatch accuracy: 85.9,	Validation accuracy: 86.6
Test accuracy: 93.0%


In [63]:
myNetwork3(0.01, 3001, 500)

Hidden nodes size = 1024
Step    0,	Minibatch accuracy: 13.3,	Validation accuracy: 28.3
Step  500,	Minibatch accuracy: 85.9,	Validation accuracy: 83.9
Step 1000,	Minibatch accuracy: 84.4,	Validation accuracy: 84.9
Step 1500,	Minibatch accuracy: 88.3,	Validation accuracy: 85.9
Step 2000,	Minibatch accuracy: 91.4,	Validation accuracy: 86.2
Step 2500,	Minibatch accuracy: 86.7,	Validation accuracy: 86.2
Step 3000,	Minibatch accuracy: 84.4,	Validation accuracy: 86.7
Test accuracy: 93.2%


In [74]:
myNetwork3(0.01, 3001, 500)

Hidden nodes size = 1024
Step    0,	Minibatch accuracy: 7.0,	Validation accuracy: 22.2
Step  500,	Minibatch accuracy: 88.3,	Validation accuracy: 84.0
Step 1000,	Minibatch accuracy: 82.0,	Validation accuracy: 84.4
Step 1500,	Minibatch accuracy: 88.3,	Validation accuracy: 85.5
Step 2000,	Minibatch accuracy: 89.8,	Validation accuracy: 85.9
Step 2500,	Minibatch accuracy: 85.9,	Validation accuracy: 85.8
Step 3000,	Minibatch accuracy: 87.5,	Validation accuracy: 86.3
Test accuracy: 92.9%


In [75]:
myNetwork3(0.01, 10001, 500)

Hidden nodes size = 1024
Step    0,	Minibatch accuracy: 3.9,	Validation accuracy: 30.8
Step  500,	Minibatch accuracy: 84.4,	Validation accuracy: 83.4
Step 1000,	Minibatch accuracy: 87.5,	Validation accuracy: 84.5
Step 1500,	Minibatch accuracy: 89.8,	Validation accuracy: 85.8
Step 2000,	Minibatch accuracy: 93.8,	Validation accuracy: 86.1
Step 2500,	Minibatch accuracy: 87.5,	Validation accuracy: 85.8
Step 3000,	Minibatch accuracy: 83.6,	Validation accuracy: 86.1
Step 3500,	Minibatch accuracy: 84.4,	Validation accuracy: 86.8
Step 4000,	Minibatch accuracy: 89.8,	Validation accuracy: 87.1
Step 4500,	Minibatch accuracy: 88.3,	Validation accuracy: 86.8
Step 5000,	Minibatch accuracy: 88.3,	Validation accuracy: 87.2
Step 5500,	Minibatch accuracy: 85.9,	Validation accuracy: 86.8
Step 6000,	Minibatch accuracy: 85.2,	Validation accuracy: 86.9
Step 6500,	Minibatch accuracy: 92.2,	Validation accuracy: 87.1
Step 7000,	Minibatch accuracy: 85.9,	Validation accuracy: 87.7
Step 7500,	Minibatch accuracy: 